In [1]:
%load_ext autoreload
%load_ext autotime

time: 173 µs (started: 2024-10-09 15:39:57 -07:00)


In [2]:
import os
import sys
import time
import warnings

# Add WisIO to the path
sys.path.append(os.path.abspath("../"))

# Filter warnings
warnings.filterwarnings('ignore')

time: 540 µs (started: 2024-10-09 15:39:57 -07:00)


In [3]:
analyzer = "dftracer"
checkpoint = True
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = f".wisio/{analyzer}/{int(time.time())}"
bottleneck_dir = f"{run_dir}/bottlenecks"
checkpoint_dir = f"{run_dir}/checkpoints"
percentile = 0.95
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/cm1/APP/node-1/v1/RAW/*.pfw.gz"

time: 734 µs (started: 2024-10-09 15:39:57 -07:00)


In [4]:
# Parameters
analyzer = "dftracer"
checkpoint = True
checkpoint_dir = "/usr/workspace/iopp/wisio_logs/_checkpoints/dftracer-bert-v100-node-16-v3"
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = ".wisio/dftracer/bert-v100-node-16-v3"
percentile = 0.99
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/bert/v100/node-16/v3/COMPACT/*.pfw.gz"


time: 668 µs (started: 2024-10-09 15:39:57 -07:00)


In [5]:
from distributed import Client
from hydra import compose, initialize
from hydra.core.hydra_config import HydraConfig
from hydra.utils import instantiate
from wisio.config import init_hydra_config_store
from wisio.utils.file_utils import ensure_dir

ensure_dir(run_dir)

with initialize(version_base=None, config_path=None):
    init_hydra_config_store()
    cfg = compose(
        config_name="config",
        overrides=[
            f"+analyzer={analyzer}",
            f"analyzer.bottleneck_dir={bottleneck_dir}",
            f"analyzer.checkpoint={checkpoint}",
            f"analyzer.checkpoint_dir={checkpoint_dir}",
            f"hydra.run.dir={run_dir}",
            f"hydra.runtime.output_dir={run_dir}",
            f"logical_view_types={logical_view_types}",
            f"output.max_bottlenecks={output_max_bottlenecks}",
            f"output.root_only={output_root_only}",
            f"percentile={percentile}",
            f"trace_path={trace_path}",
        ],
        return_hydra_config=True,
    )
    HydraConfig.instance().set_config(cfg)
    cluster = instantiate(cfg.cluster)
    client = Client(cluster)
    analyzer = instantiate(cfg.analyzer)
    output = instantiate(cfg.output)

time: 6.71 s (started: 2024-10-09 15:39:57 -07:00)


In [6]:
result = analyzer.analyze_trace(
    trace_path=cfg.trace_path,
    # accuracy=cfg.accuracy,
    exclude_bottlenecks=cfg.exclude_bottlenecks,
    exclude_characteristics=cfg.exclude_characteristics,
    logical_view_types=cfg.logical_view_types,
    metrics=cfg.metrics,
    percentile=cfg.percentile,
    view_types=cfg.view_types,
)

time: 2min 38s (started: 2024-10-09 15:40:04 -07:00)


In [7]:
output.handle_result(metrics=cfg.metrics, result=result)

╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Runtime          1029.25 seconds                                                                               │
│  I/O Time         2.57 seconds                                                                                  │
│                   ├── Read - 0.78 seconds (30.47%)                                                              │
│                   ├── Write - 0.01 seconds (0.22%)                                                              │
│                   └── Metadata - 2.38 seconds (92.60%)                                                          │
│  I/O Operations   8,690,495 ops                                                                                 │
│                   ├── Read - 287,384 ops (3.31%)                                                                │
│                   ├── Write - 4,015 ops (0.05%)                                                                 │
│                   └── Metadata - 8,363,768 ops (96.24%)                                                         │
│  I/O Size         15.46 GiB                                                                                     │
│                   ├── Read - 15.46 GiB (100.00%)                                                                │
│                   └── Write - 178.70 kiB (0.00%)                                                                │
│  Read Requests    4 kiB-256 kiB - 286,872 ops                                                                   │
│                   ├── <4 kiB - 173,976 ops (60.65%)                                                             │
│                   ├── 4-16 kiB - 20,000 ops (6.97%)                                                             │
│                   ├── 16-64 kiB - 31,456 ops (10.97%)                                                           │
│                   └── 64-256 kiB - 61,440 ops (21.42%)                                                          │
│  Write Requests   4 kiB-4 kiB - 4,015 ops                                                                       │
│                   └── <4 kiB - 4,015 ops (100.00%)                                                              │
│  Nodes            16 nodes                                                                                      │
│                   ├── 4388746569561443104 - 2.57 s (100.00%) - 0.97 GiB/10.56 kiB R/W (6.25/0.00%) - 543,133    │
│                   │   ops (6.25%)                                                                               │
│                   ├── 11986966788007745833 - 2.55 s (99.41%) - 0.97 GiB/10.56 kiB R/W (6.25/0.00%) - 543,108    │
│                   │   ops (6.25%)                                                                               │
│                   ├── 10745869406497469567 - 2.53 s (98.51%) - 0.97 GiB/10.56 kiB R/W (6.25/0.00%) - 543,151    │
│                   │   ops (6.25%)                                                                               │
│                   ├── 10315536246191266027 - 2.51 s (97.90%) - 0.97 GiB/10.56 kiB R/W (6.25/0.00%) - 543,150    │
│                   │   ops (6.25%)                                                                               │
│                   ├── 9973927505116922098 - 2.51 s (97.79%) - 0.97 GiB/10.56 kiB R/W (6.25/0.00%) - 543,148     │
│                   │   ops (6.25%)                                                                               │
│                   ├── 16065659427268196451 - 2.51 s (97.63%) - 0.97 GiB/10.56 kiB R/W (6.25/0.00%) - 543,151    │
│                   │   ops (6.25%)                                                                               │
│                   ├── 12604783666688308289 - 2.50 s (9

time: 5.26 s (started: 2024-10-09 15:42:42 -07:00)
